# Pre-canned Queries for Python
## Function to call Government and Corporate Bond Reference Data
This query demonstrates how using a list of different identifier types and identifiers, the user can request:
  * Instrument level reference data including:
  * Instrument Identifiers
  * Coupon Schedule
  * Amount Outstanding Schedule sorted in descending effective date order
  * Identify the Valuation Quote (the quote used to calculate market capitalization of the issue)
  * Quote level identifiers for all active quotes of the instrument
  * The name, TRBC Scheme level 5, organization type, status, and ultimate parent ID for the issuer of the instrument
The query demonstrates how fields that contain enumerated values can be configured to return a human readable value rather than the enumeration Entitlements: To access the data returned by this query, the user account must have licenses that include:
  * Government and Corporate Bond Reference Data - PRS / FI_GOV_REFERENCE
  * Organization Reference Data - PRS / BASIC_ENTITY
The query requests instrument identifiers associated with the instrument - if the instrument being queried  has a SEDOL or CUSIP, the Identifier will be suppressed unless the user has the appropriate licenses (SYMBOLOGY / IDENTIFIER_TYPE_SEDOL and IDENTIFER_TYPE_SEDOL)


tags :     `Government`, `Corporate`, `Reference`, `Instrument`, `Organization`, `Identifiers`
## Import the library and load credentials
Credentials used by this notebook are stored in the config_files/credentials.ipynb. Please edit credentials.ipynb to set your credentials and run the next cell to continue with the scripts 

In [ ]:
%run ../../../config_files/credentials.ipynb 
%run ../../../config_files/environment_selection.ipynb

To connect to the Refinitiv Data and opening a session,given function is used. To find out more about the library,you can visit [refinitiv-data library](https://pypi.org/project/refinitiv-data/)

In [ ]:
session = rd.session.platform.Definition(
    app_key=APP_KEY,
    grant=rd.session.platform.GrantPassword(
        username=RD_LOGIN, password=RD_PASSWORD
    ),
).get_session()
session.open()


## Graph call
Graph call with given sets of samples below. 

In [ ]:
%%writefile query_cache
query GovernmentCorporateBondReferenceData($InstrumentList: [GovCorpInstrumentIdentifierArgs!]) {
  GovCorpInstrument(identifiers: $InstrumentList) {
    ObjectId
    _metadata {
      identifierTypeName
      identifier
    }
    Reference {
      SecurityDescription
      MaturityDate
      CouponTypeCode
      CouponTypeCodeDescription: CouponTypeCode(format: Name, language: en_US, style: LongName)
      Cic
      CfiCode
    }
    AssetStatus {
      AssetStatusCode
      AssetStatusCodeDescription: AssetStatusCode(format: Name, language: en_US, style: LongName)
      EffectiveFromDate
    }
    CouponRateSchedule {
      CouponRate
      EffectiveFromDate
      InterestLegNumber
      LastChangedDate
    }
    IssuanceDefinition {
      DatedDate
      IssueCurrencyIsoCode
      IssueAmount
    }
    InterestDefinition {
      DayCountCode_v2
      DayCountCodeDescription: DayCountCode_v2(format: Name, language: en_US, style: LongName)
    }
    AmountOutstandingHistory(sort: {EffectiveFromDate: DESC}) {
      CurrencyIsoCode
      TotalOutstandingAmount
      EffectiveFromDate
      NetChangeCode
      NetChangeCodeDescription: NetChangeCode(format: Name, language: en_US, style: LongName)
    }
    Identifiers {
      EffectiveFromDate
      IdentifierCode
      IdentifierCodeDescription: IdentifierCode(format: Name, language: en_US, style: LongName)
      IdentifierValue
    }
    Quotes {
      ObjectId
      Identifier(filter: {AND: [{IdentifierTypeCode: {IN: ["ORC", "RIC"]}}, {QuoteIsActiveFlag: {EQ: "y"}}]}) {
        GlobalRegionCode
        IdentifierTypeCode
        IdentifierValue
        ObjectTypeCode
        QuoteContributorCode
        QuoteCurrencyIsoCode
        QuoteIsActiveFlag
        QuotePriceSourceCode
        QuotePriceSourceCodeDescription: QuotePriceSourceCode(format: Name, language: en_US, style: LongName)
      }
    }
    Issuer {
      ObjectId
      Names {
        Name {
          OrganizationName(filter: {NameTypeCode: {EQ: "LNG"}}) {
            Name
            NameTypeCode
            NameTypeCodeDescription: NameTypeCode(format: Name, language: en_US, style: LongName)
          }
        }
      }
      ClassificationTrbcPrimaryScheme_v2 {
        Industry {
          Name
        }
        Identifier {
          Identifiers(filter: {ValueTypeId: {EQ: "320159"}}) {
            Value
            ValueTypeCode
            ValueTypeId
          }
        }
      }
      Details {
        OrganizationDetails {
          JurisdictionOfIncorporation
          OrganizationTier
          OrganizationTypeCode(format: Name, language: en_US, style: LongName)
          OrganizationSubTypeCode(format: Name, language: en_US, style: LongName)
          OrganizationStatusCode
          OrganizationStatusCodeDescription: OrganizationStatusCode(format: Name, language: en_US, style: LongName)
        }
      }
      UltimateParentId
    }
  }
}


## Sample :  Find by a mixture of Identifier Types

In [ ]:

variables = {'InstrumentList': [{'identifierTypeName': 'Isin', 'identifier': 'US645767AW49'}, {'identifierTypeName': 'Sedol', 'identifier': 'B4333T3'}, {'identifierTypeName': 'Cusip', 'identifier': '36966TDN9'}]}

In [3]:
output_format = 'handlebars'  # available: handlebars
handlebars_template = 'Government and Corporate Bond Reference Data.html'
%run ../../../config_files/config_request_handler.ipynb

<table>
        <tbody>
        <tr><td colspan="2"><b>Identifier Type</b></td></tr>
        <tr>
            <td>Identifier Type Name</td>
            <td>Cusip</td>
        </tr>
        <tr>
            <td>Identifier</td>
            <td>36966TDN9</td>
        </tr>
        <tr><td colspan="2"><b>Reference</b></td></tr>
        <tr>
            <td>Security Description</td>
            <td>GE     4.300 11/15/25 MTN</td>
        </tr>
        <tr>
            <td>Maturity Date</td>
            <td>2025-11-15</td>
        </tr>
        <tr>
            <td>Coupon Type Code</td>
            <td>FXPV</td>
        </tr>
        <tr>
            <td>Coupon Type Code Description</td>
            <td>Fixed:Plain Vanilla Fixed Coupon</td>
        </tr>
        <tr>
            <td>Cic</td>
            <td>US21</td>
        </tr>
        <tr>
            <td>Cfi Code</td>
            <td>DTFUFR</td>
        </tr>
        <tr><td colspan="2"><b>Asset Status</b></td></tr>
        <tr>
            <td>Asset Status Code</td>
            <td>ISS</td>
        </tr>
        <tr>
            <td>Asset Status Code Description</td>
            <td>Issued</td>
        </tr>
        <tr>
            <td>Asset Effective From Date</td>
            <td>2011-11-03</td>
        </tr>
        <tr><td colspan="2"><b>Coupon Rate Schedule</b></td></tr>
        <tr>
            <td>Coupon Rate</td>
            <td>4.3</td>
        </tr>
        <tr>
            <td>Coupon Effective From Date</td>
            <td>2011-11-03</td>
        </tr>
        <tr>
            <td>Coupon Interest Leg Number</td>
            <td>1</td>
        </tr>
        <tr>
            <td>Coupon Last Changed Date</td>
            <td>2020-04-19T23:44:34.063Z</td>
        </tr>
        <tr><td colspan="2"><b>Issuance Definition</b></td></tr>
        <tr>
            <td>Dated Date</td>
            <td>2011-11-03</td>
        </tr>
        <tr>
            <td>Issue Currency Iso Code</td>
            <td>USD</td>
        </tr>
        <tr>
            <td>Issue Amount</td>
            <td>16024</td>
        </tr>
        <tr><td colspan="2"><b>Interest Definition</b></td></tr>
        <tr>
            <td>Day Count Code</td>
            <td>1</td>
        </tr>
        <tr>
            <td>Day Count Code Description</td>
            <td>30/360 US, 30U/360, 30US/360</td>
        </tr>
        <tr><td colspan="2"><b>Amount Outstanding History</b></td></tr>
        <tr>
            <td>Currency Iso Code</td>
            <td>USD</td>
        </tr>
        <tr>
            <td>Total Outstanding Amount</td>
            <td>16024</td>
        </tr>
        <tr>
            <td>Effective From Date</td>
            <td>2011-11-03</td>
        </tr>
        <tr>
            <td>Net Change Code</td>
            <td>ISS</td>
        </tr>
        <tr>
            <td>Net Change Code Description</td>
            <td>Initial Issuance</td>
        </tr>
        <tr>
            <td colspan="2"><b>Identifiers</b></td>
        </tr>
        <tr>
            <td style="width: 50%;">
                <ul>
                    <li>Effective from date</li>
                    <li>Identifier Code</li>
                    <li>Identifier Code Description</li>
                    <li>Identifier Value</li>
                </ul>
            </td>
            <td style="width: 50%;">
                <ul>
                    <li>2014-07-15</li>
                    <li>NST</li>
                    <li>NASD Bond Symbol</li>
                    <li>GE.ATI</li>
                </ul>
            </td>
        </tr>
        <tr>
            <td colspan="2"><b>Quotes</b></td>
        </tr>
        <tr>
            <td style="width: 50%;">
                <ul>
                    <li>Global Region Code</li>
                    <li>Identifier Type Code</li>
                    <li>Identifier Value</li>
                    <li>Object Type Code</li>
                    <li>Quote Contributor Code</li>
                   

## Close the default session when done

In [ ]:
session.close()